In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import itertools
import math
from matplotlib.ticker import AutoMinorLocator
from functools import reduce

f_alg = {
    "SPM": "Small Progress Measures",
    "SCC_FB": "Forward-Backward",
    "SCC_COL": "Colouring/Heads off",
    "synthesis": "Supervisory Controller Synthesis",
    "prefix_sum": "Prefix Sum"
};

f_apt = {
    ("prefix_sum", "Random"): "Prefix Sum",
    ("SCC_FB", "(P=1.3/N)"): "FB",
    ("SCC_COL", "(P=1.3/N)"): "CH",
    ("synthesis", "Random"): "SCS",
    ("SPM", "Invariantly Inevitably Eat"): "SPM IIE",
    ("SPM", "Invariantly Plato Starves"): "SPM IPS",
};

f_order = {
    "relaxed": "Relaxed",
    "acqrel": "RA",
    "seqcons": "SC"
};

f_schedule = {
    "in-kernel": "In-kernel",
    "on-host": "On-host",
    "graph": "Graph"
};

order_color = {
    "relaxed": "C0",
    "acqrel": "C2",
    "seqcons": "C1"
};

apt_marker = {
    ("prefix_sum", "Random"): "P",
    ("SCC_FB", "(P=1.3/N)"): "4",
    ("SCC_COL", "(P=1.3/N)"): "x",
    ("synthesis", "Random"): "v",
    ("SPM", "Invariantly Inevitably Eat"): ".",
    ("SPM", "Invariantly Plato Starves"): "*",
};

alg_marker = {
    "SPM": ".",
    "SCC_FB": "4",
    "SCC_COL": "x",
    "synthesis": "v",
    "prefix_sum": "P"
};

opt_linestyles = {
    True: "-",
    False: (0, (1, 3))
};
    


algs_ptypes = [
    ("prefix_sum", "Random"),
    ("SCC_FB", "(P=1.3/N)"),
    ("SCC_COL", "(P=1.3/N)"),
    ("synthesis", "Random"),
    ("SPM", "Invariantly Inevitably Eat"),
    ("SPM", "Invariantly Plato Starves"),
];

def cols2rt(s, o, opt, v):
    return f"rt_{s}_{o}_{opt}_{v}";

def max_threads(s, alg):
    mt_per_alg_ik = {
        "SPM": 58880,
        "SCC_COL": 58880,
        "prefix_sum": 70656,
        "synthesis": 70656,
        "SCC_FB": 52992
    };
    if s == "graph" or s == "on-host":
        return 70656;
    else:
        return mt_per_alg_ik[alg];

threads_color = "black";
threads_ls = "--";#(0, (4, 8));
threads_alpha = 0.5;

# AuDaLa data
spm = pd.read_csv('SPM/SPM_results.csv', sep=',')
scc_col = pd.read_csv('SCC/SCC_COL_results.csv', sep=',')
scc_fb = pd.read_csv('SCC/SCC_FB_results.csv', sep=',')
#sorting = pd.read_csv('sorting/sorting_results.csv', sep=',')
prefix_sum = pd.read_csv('prefix_sum/prefix_sum_results.csv', sep=',')
synthesis = pd.read_csv('synthesis/synthesis_results.csv', sep=',')

all_data = [spm, scc_col, scc_fb, prefix_sum, synthesis];
cols = list(map((lambda d : d.columns), all_data));
assert (all([all(cols[i-1] == cols[i]) for i in range(1, len(cols))]));

data = pd.concat(all_data, ignore_index=True);
data = data.drop(data[data["runtime"] == "timeout"].index);
data["runtime"] = data["runtime"].astype(float);
data["problem_size1"] = data["problem_size1"].astype(int);
data["problem_size2"] = data["problem_size2"].astype(int);
data["problem_size3"] = data["problem_size3"].astype(int);
data["weak_non_racing"] = data["weak_non_racing"].astype(bool);

data.loc[:, "problem_size"] = data[["problem_size1", "problem_size2", "problem_size3"]].max(axis=1);


# Sequential data
synthesis_seq = pd.read_csv('synthesis/synthesis_sequential.csv', sep=',')
spm_seq = pd.read_csv('SPM/SPM_sequential.csv', sep=',')
prefix_sum_seq = pd.read_csv('prefix_sum/prefix_sum_sequential.csv', sep=',')
scc_seq = pd.read_csv('SCC/scc_sequential.csv', sep=',')

all_data = [synthesis_seq, spm_seq, prefix_sum_seq, scc_seq];
cols = list(map((lambda d : d.columns), all_data));
assert (all([all(cols[i-1] == cols[i]) for i in range(1, len(cols))]));
seq_data = pd.concat(all_data, ignore_index=True);
seq_data["runtime"] = seq_data["runtime"].astype(float);
seq_data["problem_size1"] = seq_data["problem_size1"].astype(int);
seq_data["problem_size2"] = seq_data["problem_size2"].astype(int);
seq_data["problem_size3"] = seq_data["problem_size3"].astype(int);
seq_data["problem_size"] = seq_data[["problem_size1", "problem_size2", "problem_size3"]].max(axis=1);


# Make df with (problem_size, algorithm, problem_type) as index and various runtimes in columns
bench_columns = list(data.groupby(["schedule", "memorder", "weak_non_racing", "voting-strat"]).groups.keys());
memory_orders = list(data["memorder"].unique());

prepped = pd.concat([
    data[
        (data["memorder"] == o) &
        (data["schedule"] == s) &
        (data["voting-strat"] == v) &
        (data["weak_non_racing"] == opt)
    ][["problem_size", "algorithm", "problem_type", "runtime"]]
    .set_index(["problem_size", "algorithm", "problem_type"])
    .rename(columns={"runtime": cols2rt(s,o,opt,v)})
    for (s, o, opt, v) in bench_columns
], axis = 1);
prepped.sort_index(inplace=True);
prepped[prepped.index.get_level_values('algorithm') == "SCC_FB"][["rt_in-kernel_relaxed_True_in-kernel-simple", "rt_in-kernel_relaxed_True_in-kernel-alternating"]]

In [ ]:
rel_color = "#191970";
sc_color = "#800000"; 


def seqcmp_chart(rows, algs_ptypes):
    marker = "x";
    fig, ax = plt.subplots(nrows=1, ncols=1, constrained_layout=True, figsize=(6, 4));
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.grid(True, zorder=0)
    ax.set_ylabel("Sequential runtime (ms)", fontsize=16)
    ax.set_xlabel("AuDaLa runtime (ms)", fontsize=16)
    ax.axline((0, 0), (1, 1), label="x = y", linewidth=0.75, color=sc_color)
    ax.tick_params(axis='x', which='minor', bottom=False);
    ax.tick_params(axis='y', which='minor', left=False);
    
    adl = cols2rt("graph", "relaxed", True, "graph-simple");
    for (a, pt) in algs_ptypes:
        adl_rt = rows.loc[
            (rows.index.get_level_values('algorithm') == a) &
            (rows.index.get_level_values('problem_type') == pt)
            , :
        ][adl];
        seq_rt = seq_data[(seq_data["alg"] == a) & (seq_data["problem_type"] == pt)]["runtime"];
        
        if pt == "Invariantly Inevitably Eat":
            art = adl_rt[:-2];
            srt = seq_rt[:-2];
#         elif a == "prefix_sum":
#             art = adl_rt[3:];
#             srt = seq_rt[3:];
        else:
            art = adl_rt;
            srt = seq_rt;
        
        ax.plot(
            art,
            srt,
            color=rel_color,
            linestyle=(0, (5, 10)),
            linewidth=0.5,
            marker=apt_marker[(a, pt)],
            label=f"{f_apt[(a, pt)]}"
        );

    fig.legend(bbox_to_anchor=(1.28, 1.01), fontsize=12)
    plt.savefig(f"seq-cmp.svg", bbox_inches = 'tight');
    plt.show()
    

apt = algs_ptypes.copy();
#apt.remove(("SPM", "Invariantly Inevitably Eat"));
seqcmp_chart(prepped, apt);


In [ ]:
def voting_strat_chart(rows, algs_ptypes):
    marker = "x";
    fig, axs = plt.subplots(nrows=1, ncols=1, sharey=True,constrained_layout=True, figsize=(6, 4));

    cols = {
        "relaxed": axs,
        "seqcons": axs
    };
    
    for (s, o, opt, v) in bench_columns:
        if o != "relaxed":
            continue;
        if not opt:
            continue;
        if "alternating" in v or "graph" in v:
            continue;
        
        assert(v.endswith("-simple"))
        color = sc_color if s == "on-host" else rel_color;
        rt_col = cols2rt(s, o, opt, v);
        rt_col_cmp = cols2rt(s, o, opt, v[:-7] + "-alternating");
        
        the_ax = cols[o];
        
        for (a, pt) in algs_ptypes:
            alg_rows = rows.loc[
                (rows.index.get_level_values('algorithm') == a) &
                (rows.index.get_level_values('problem_type') == pt)
                , :
            ];
            if a == "SCC_COL":
                the_ax.plot(
                    alg_rows.index.get_level_values('problem_size'),
                    alg_rows[rt_col] / alg_rows[rt_col_cmp],
                    color=color,
                    linestyle=(0, (1, 4)),
                    linewidth=1,
                    marker=apt_marker[(a, pt)],
                    label=f"{f_schedule[s]} ({f_apt[(a,pt)]})",
                    zorder=3+(2 if s == "on-host" else 0)
                )
            else:
                the_ax.scatter(
                    alg_rows.index.get_level_values('problem_size'),
                    alg_rows[rt_col] / alg_rows[rt_col_cmp],
                    color=color,
                    marker=apt_marker[(a, pt)],
                    label=f"{f_schedule[s]} ({f_apt[(a,pt)]})",
                    linewidths=0.8,
                    zorder=3+(2 if s == "on-host" else 0)
                )

        the_ax.set_xscale("log")
        the_ax.grid(True, zorder=0)
        the_ax.set_xlabel("Problem size", fontsize=16)
        the_ax.set_ylabel("Slowdown over Rotating", fontsize=16)
        the_ax.tick_params(axis='x', which='minor', bottom=False);
            
        the_ax.set_xlim(10**2.9, 10**7.2);

    fig.legend(bbox_to_anchor=(1.43, 1.01), fontsize=12)
    #fig.legend(handles=patches, bbox_to_anchor=(1.25, 1.01), fontsize=12)
    plt.savefig(f"voting-strat-impact.svg", bbox_inches = 'tight');
    plt.show()

voting_strat_chart(prepped, algs_ptypes);

In [ ]:
def schedule_strat_chart(rows, algs_ptypes):
    marker = "x";
    fig, ax = plt.subplots(nrows=1, ncols=1, sharey=True,constrained_layout=True, figsize=(6, 4));
    
    for (s, o, opt, v) in bench_columns:
        if o != "relaxed":
            continue;
        if not opt:
            continue;
        if "graph" in v or "simple" in v:
            continue;
        assert(v.endswith("-alternating"))
        
        color = sc_color if s == "on-host" else rel_color;
        rt_col = cols2rt(s, o, opt, v);
        rt_col_cmp = cols2rt("graph", o, opt, "graph-simple");
        for (a, pt) in algs_ptypes:
            alg_rows = rows.loc[
                (rows.index.get_level_values('algorithm') == a) &
                (rows.index.get_level_values('problem_type') == pt)
                , :
            ];
            ax.scatter(
                alg_rows.index.get_level_values('problem_size'),
                alg_rows[rt_col] / alg_rows[rt_col_cmp],
                color=color,
                marker=apt_marker[(a, pt)],
                label=f"{f_schedule[s]} ({f_apt[(a,pt)]})",
                linewidths=0.8,
                zorder=3+(2 if s == "on-host" else 0)
            )
        
        ax.set_xticks([10**(3+n) for n in range(5)]);
        ax.set_xticklabels([f"$10^{3+n}$" for n in range(5)]);
        ax.tick_params(axis='x', which='minor', bottom=False);
        ax.set_xscale("log")
        
        ax.set_yticks([k for k in range(1, 9)]);
        ax.set_yticklabels([str(k) for k in range(1, 9)]);
        ax.tick_params(axis='y', which='minor', left=False);
        #ax.set_yscale("log")
        
        ax.grid(True, zorder=0)
        ax.set_xlabel("Problem size", fontsize=16)
        ax.set_ylabel("Slowdown over Graph", fontsize=16)
            
        ax.set_xlim(10**2.9, 10**7.2);
    #ax.axvline(max_threads("graph", "_"), color=threads_color, alpha=threads_alpha, linestyle=threads_ls, label="Max threads");
    fig.legend(bbox_to_anchor=(1.43, 1.01), fontsize=12)
    plt.savefig(f"schedule-strat-impact.svg", bbox_inches = 'tight');
    plt.show()

schedule_strat_chart(prepped, algs_ptypes);

In [ ]:


def onrp_scatter_chart(rows, algs_ptypes):
    marker = "x";
    
    #fig = plt.figure(constrained_layout=True, figsize=(12, 4));
    fig, axs = plt.subplots(nrows=1, ncols=1, sharey=True,constrained_layout=True, figsize=(6, 4));
#     axs[1].yaxis.set_label_position("right")
#     axs[1].yaxis.tick_right()

    cols = {
        "relaxed": axs,#[1],
        "seqcons": axs#[0],
    };
    
    for (s, o, opt, v) in bench_columns:
        if opt:
            continue;
        if not v.endswith("simple"):
            continue;
        if o == "acqrel":
            continue;

        color = rel_color if o == "relaxed" else sc_color;
        rt_col = cols2rt(s, o, False, v);
        rt_col_cmp = cols2rt(s, o, True, v);
        
        the_ax = cols[o];
        
        for (a, pt) in algs_ptypes:
            alg_rows = rows.loc[
                (rows.index.get_level_values('algorithm') == a) &
                (rows.index.get_level_values('problem_type') == pt)
                , :
            ];
            the_ax.scatter(
                alg_rows.index.get_level_values('problem_size'),
                alg_rows[rt_col] / alg_rows[rt_col_cmp],
                color=color,
                marker="x" if o == "relaxed" else ".",
                #s=12,
                linewidths=0.8,
                zorder=3+(2 if o == "relaxed" else 0)
            )

        the_ax.set_xscale("log")
        the_ax.grid(True, zorder=0)
        the_ax.set_xlabel("Problem size", fontsize=16)
        the_ax.set_ylabel("Slowdown over ONRP", fontsize=16)
        the_ax.set_xlim(10**2.9, 10**7.2);
        the_ax.tick_params(axis='x', which='minor', bottom=False);
    
    patches = [
        Line2D(
            [0],
            [0],
            color=rel_color,
            label=f"Relaxed",
            marker=marker,
            linestyle="None"
        ),
        Line2D(
            [0],
            [0],
            color=sc_color,
            label=f"SC",
            marker="x" if o == "relaxed" else ".",
            linestyle="None"
        )
    ];
    
    fig.legend(handles=patches, bbox_to_anchor=(1.25, 1.01), fontsize=12)
    plt.savefig(f"ONRP-impact.svg", bbox_inches = 'tight');
    plt.show()

onrp_scatter_chart(prepped, algs_ptypes);

In [ ]:
# returns (color, marker, linestyle, line-alpha)
style = {
#     ("acqrel", True) : ("#191970", "x", "--", 0.7),
#     ("acqrel", False) : ("#191970", "+", ":", 1),
#     ("seqcons", True) : ("#b50709", "x", "--", 0.7),
#     ("seqcons", False) : ("#b50709", "+", ":", 1),
    ("acqrel", True) : (rel_color, "x", "-", 1),
    ("acqrel", False) : (rel_color, "+", ":", 1),
    ("seqcons", True) : (sc_color, "x", "-", 1),
    ("seqcons", False) : (sc_color, "+", ":", 1),
}


def memorder_chart(rows, algs_ptypes, ONRP):
    schedules = ["in-kernel", "on-host", "graph"];
    def row(alg_pt):
        return algs_ptypes.index(alg_pt);
    
    def col(s):
        return schedules.index(s);
    
    fig = plt.figure(constrained_layout=True, figsize=(12, 3 * len(algs_ptypes)));
    subfigs_rows = fig.subfigures(nrows=len(algs_ptypes), ncols=1)
    if len(algs_ptypes) == 1:
        subfigs_rows = [subfigs_rows];
    axs = [subfigs_rows[r].subplots(nrows=1, ncols=3, sharey=True) for r in range(len(algs_ptypes))];
    
    for (r, sf) in enumerate(subfigs_rows):
        alg = algs_ptypes[r][0];
        p_type = algs_ptypes[r][1];
        name = f_alg[alg];
        sub_name = f" ({p_type})" if alg == "SPM" else "";
        sf.suptitle(f'{name}{sub_name}:', fontsize=15, fontweight="bold");
    
    for (r, ax_row) in enumerate(axs):
        for (c, ax) in enumerate(ax_row):
            ax.set_xlim(10**2.9, 10**7.2);
            ax.grid(True);
            ax.set_xticks([10**(3+n) for n in range(5)]);
            ax.set_xticklabels([f"$10^{3+n}$" for n in range(5)]);
            ax.tick_params(axis='x', which='minor', bottom=False);
            ax.set_xscale("log");
            
            ax.axvline(max_threads(schedules[c], algs_ptypes[r][0]), color=threads_color, alpha=threads_alpha, linestyle=threads_ls);
            if c == 0:
                ax.set_ylabel("Slowdown over Relaxed", fontsize=12)
            if r == 0:
                ax.set_title(f_schedule[schedules[c]], fontsize=16);
            if r == len(axs) - 1:
                ax.set_xlabel("Problem size", fontsize=16)
        
    for ((a, pt), (s, o, opt, v)) in itertools.product(algs_ptypes, bench_columns):
        if opt not in ONRP:
            continue;
        if o == "relaxed":
            continue;
        # Already implied by order == relaxed
        if v.endswith("simple") == False:
            continue;
        
        c = col(s);
        r = row((a, pt));
        ax = axs[r][c];
        alg_rows = rows.loc[
            (rows.index.get_level_values('algorithm') == a) &
            (rows.index.get_level_values('problem_type') == pt)
            , :
        ];
        
        rt_col = cols2rt(s, o, opt, v);
        rt_col_cmp = cols2rt(s, "relaxed", opt, v);
        
        (color, marker, linestyle, line_alpha) = style[(o, opt)];
        ax.scatter(
            alg_rows.index.get_level_values('problem_size'),
            alg_rows[rt_col] / alg_rows[rt_col_cmp],
            color=color,
            zorder=3,
            marker=marker
        )
        ax.plot(
            alg_rows.index.get_level_values('problem_size'),
            alg_rows[rt_col] / alg_rows[rt_col_cmp],
            color=color,
            zorder=3,
            linestyle=linestyle, 
            alpha=line_alpha
        )

    patches = [
        Line2D(
            [0],
            [0],
            color=style[(o, opt)][0],
            linestyle=style[(o, opt)][2],
            label=f_order[o],
            linewidth=1,
            marker=style[(o, opt)][1]
        ) for (o, opt) in itertools.product(["seqcons", "acqrel"], ONRP)
    ];
    
    patches.append(Line2D([0], [0], color=threads_color, alpha=threads_alpha, linestyle=threads_ls, label="Max threads"))
    
    fig.legend(handles=patches, bbox_to_anchor=(1.17, 0.969), fontsize=14);
    
    plt.savefig(f"memorder-impact.svg", bbox_inches = 'tight');
    plt.show();

apt = algs_ptypes.copy();
apt.remove(("prefix_sum", "Random"));
memorder_chart(prepped, apt, [True])

In [ ]:
def schedule_chart(alg, p_type, data, seq_data, relative = False, per_psize = False, schedules = None):
    assert(not(relative and per_psize));
    
    rows = data[(data["algorithm"] == alg) & (data["problem_type"] == p_type)];
    if schedules is not None:
        rows = rows[rows["schedule"].isin(schedules)];

    voting = sorted(list(set(rows.groupby(["schedule", "voting-strat"]).groups.keys())));
    voting.remove(("graph", "graph-shared"))
    voting.remove(("graph", "graph-shared-banks"))
    #voting = [("in-kernel", "in-kernel-simple")];
    prepped = pd.concat([
        rows[
            (rows["schedule"] == s) &
            (rows["voting-strat"] == v) &
            (rows["memorder"] == o) &
            (rows["weak_non_racing"] == True)
        ][["problem_size", "runtime"]]
        .set_index("problem_size")
        .sort_index()
        .rename(columns={"runtime": f"rt_{s}_{v}_{o}"})
        for ((s, v), o) in itertools.product(voting, ["relaxed"])
    ], axis = 1);
    
    colors = {
        "graph": "C0",
        "on-host": "C1",
        "in-kernel": "C2",
    };
    
    marker = {
        "graph-simple": ("*", 8, "C3"),
        "in-kernel-simple": ("*", 8, "C4"),
        "on-host-simple": ("*", 8, "C5"),
        "on-host-alternating": ("D", 5, "C6"),
        "in-kernel-alternating": ("D", 5, "C7"),
        "graph-shared": ("+", 8, "C8"),
        "graph-shared-banks": ("x", 8, "C9"),
        
    };
    
    fig, ax = plt.subplots(figsize=(6, 4));
    
    rel = ' relative' if relative else '';
    per_instance = ' per instance' if per_psize else '';
    if relative:
        ax.set_ylabel(f"Runtime relative to graph-simple");
    else:
        ax.set_ylabel(f"Runtime{per_instance} (ms)");
        ax.set_yscale("log");
    
    ax.grid(True);
    ax.set_xlim(10**3, 10**7.15);
    ax.set_xscale("log");
    ax.set_xlabel("Problem size");
    
    for ((s, v), o) in itertools.product(voting, ["relaxed"]):
        if per_psize:
            Y = prepped[f"rt_{s}_{v}_{o}"] / prepped.index;
        elif relative:
            Y = (prepped[f"rt_{s}_{v}_{o}"] / prepped[f"rt_graph_graph-simple_{o}"]) * 100;
        else:
            Y = prepped[f"rt_{s}_{v}_{o}"];

        (m, ms, mc) = marker[v];
        ax.plot(
            prepped.index,
            Y,
            label=f"{v}",
            color=colors[s],
            marker=m,
            markersize=ms,
            linestyle='--',
            markerfacecolor=mc,
            markeredgecolor=mc
        );
    
    s_data = seq_data[(seq_data["alg"] == alg) & (seq_data["problem_type"] == p_type)];
    if len(s_data.index) > 0:
        ax.plot(
            s_data["problem_size"],
            s_data["runtime"] if not per_psize else s_data["runtime"] / s_data["problem_size"],
            label="Sequential",
            color="C10"
        );
    
    ax.axvline(max_threads("graph", "."), color=threads_color, alpha=threads_alpha, linestyle=threads_ls, label="Max threads");
    
    plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left");
    ptype_small = "" if alg != "SPM" else "iie" if p_type == "Invariantly Inevitably Eat" else "ips";
    #plt.savefig(f"schedule-impact-{alg}{ptype_small}.svg", bbox_inches = 'tight');
    plt.show();

In [ ]:
for (alg, p_type) in algs_ptypes:
    print(alg);
    schedule_chart(alg, p_type, data, seq_data, relative=False, per_psize=False);


In [ ]:
def profile_chart(rows):
    fig, ax = plt.subplots(figsize=(6, 4));
    #fig = plt.figure(constrained_layout=True, figsize=(12, 3 * len(algs_ptypes)));
    
    ax.set_xlim(10**2.9, 10**7.2);
    #ax.grid(True);
    ax.set_xticks([10**(3+n) for n in range(5)]);
    ax.set_xticklabels([f"$10^{3+n}$" for n in range(5)]);
    ax.set_xscale("log");
    ax.tick_params(axis='x', which='minor', bottom=False);
    ax.set_ylabel("Slowdown over Relaxed", fontsize=14)
    ax.set_xlabel("Problem size", fontsize=14)
    
    alg_rows = rows.loc[(rows.index.get_level_values('algorithm') == "SCC_COL"), :];
    rt_col = cols2rt("graph", "seqcons", True, "graph-simple");
    rt_col_cmp = cols2rt("graph", "relaxed", True, "graph-simple");

    (color, marker, linestyle, line_alpha) = style[("seqcons", True)];
    ax.plot(
        alg_rows.index.get_level_values('problem_size'),
        alg_rows[rt_col] / alg_rows[rt_col_cmp],
        color=color,
        zorder=3,
        linestyle=linestyle, 
        alpha=0.35,
        label="SC",
        marker='',
    )
    
    prof_data = pd.read_csv('SCC/profile_results.csv', sep='\t');
    ax2 = ax.twinx();
    ax2.set_ylabel("%", fontsize=14)
    
    rel_rows = prof_data[prof_data["Memorder"] == "relaxed"];
    sc_rows = prof_data[prof_data["Memorder"] == "seqcons"];
    
    ax2.plot(
        rel_rows["problem_size"],
        rel_rows["L2 Hit Rate"],
        label=f"L2 Hit Rate (Relaxed)",
        color=rel_color,
        marker='x',
        #linestyle="--"
    );
    
    ax2.plot(
        sc_rows["problem_size"],
        sc_rows["L2 Hit Rate"],
        label=f"L2 Hit Rate (SC)",
        color=sc_color,
        marker='x',
        #linestyle="--"
    );
    ax2.grid(True)
#     ax2.plot(
#         rel_rows["problem_size"],
#         rel_rows["No Eligible"],
#         label=f"Stall rate (Relaxed)",
#         color=rel_color,
#         marker='x',
#         linestyle=":"
#     );
    
#     ax2.plot(
#         sc_rows["problem_size"],
#         sc_rows["No Eligible"],
#         label=f"Stall rate (SC)",
#         color=sc_color,
#         marker='x',
#         linestyle=":"
#     );
    ax.axvline(max_threads("graph", "_"), color=threads_color, alpha=threads_alpha, linestyle=threads_ls, label="Max threads");

    fig.legend( bbox_to_anchor=(0.48, 0.37));
    
    plt.savefig(f"profile-graph-col.svg", bbox_inches = 'tight');
    plt.show();
    
profile_chart(prepped)

In [ ]:
def profile_chart2(rows):
    fig, ax = plt.subplots(figsize=(6, 4));
    #fig = plt.figure(constrained_layout=True, figsize=(12, 3 * len(algs_ptypes)));
    
    ax.set_xlim(10**2.9, 10**7.2);
    ax.grid(True);
    ax.set_xticks([10**(3+n) for n in range(5)]);
    ax.set_xticklabels([f"$10^{3+n}$" for n in range(5)]);
    ax.set_xscale("log");
    ax.tick_params(axis='x', which='minor', bottom=False);
    ax.axvline(max_threads("graph", "_"), color=threads_color, alpha=threads_alpha, linestyle=threads_ls);
    ax.set_ylabel("Speedup")
    ax.set_xlabel("Problem size")
    
    alg_rows = rows.loc[
        (rows.index.get_level_values('algorithm') == "SPM") & 
        (rows.index.get_level_values('problem_type') == "Invariantly Plato Starves")
    , :];
    rt_col = cols2rt("graph", "seqcons", True, "graph-simple");
    rt_col_cmp = cols2rt("graph", "relaxed", True, "graph-simple");

    (color, marker, linestyle, line_alpha) = style[("seqcons", True)];
    ax.plot(
        alg_rows.index.get_level_values('problem_size'),
        alg_rows[rt_col] / alg_rows[rt_col_cmp],
        color=color,
        zorder=3,
        linestyle=linestyle, 
        alpha=line_alpha,
        label="Relaxed speedup over Seqcons"
    )
    
    prof_data = pd.read_csv('SPM/profile_SPM_results.csv', sep='\t');
    ax2 = ax.twinx();
    ax2.set_ylabel("%")
    
    for o in ["seqcons", "relaxed"]:
        o_rows = prof_data[prof_data["Memorder"] == o];
        ax2.plot(
            o_rows["problem_size"],
            o_rows["L2 Hit Rate"],
            label=f"{o} L2 Hit Rate",
            marker='x'
        );
        ax2.plot(
            o_rows["problem_size"],
            o_rows["Max Bandwidth"],
            label=f"{o} Max Bandwidth"
        );
        ax2.plot(
            o_rows["problem_size"],
            o_rows["No Eligible"],
            label=f"{o} No Eligible"
        );
    psizes = list(prof_data[prof_data["Memorder"] == "relaxed"]["problem_size"]);
    noeli_sc = list(prof_data[prof_data["Memorder"] == "seqcons"]["No Eligible"]);
    noeli_rlx = list(prof_data[prof_data["Memorder"] == "relaxed"]["No Eligible"]);
    noeli = [(a/b) for (a, b) in zip(noeli_sc, noeli_rlx)];
    
    wcpii_sc = list(prof_data[prof_data["Memorder"] == "seqcons"]["WCPII"]);
    wcpii_rlx = list(prof_data[prof_data["Memorder"] == "relaxed"]["WCPII"]);
    wcpii = [(a/b) for (a, b) in zip(wcpii_sc, wcpii_rlx)];
#     ax.plot(
#         psizes,
#         noeli,
#         label=f"Noeli"
#     );
#         ax2.plot(
#             o_rows["problem_size"],
#             o_rows["Stall Membar"],
#             label=f"{o} Stall Membar",
#             marker='x'
#         );
        
#     patches = [
#         Line2D(
#             [0],
#             [0],
#             color=color,
#             linestyle=linestyle,
#             label="Relaxed speedup over Seqcons",
#             linewidth=1,
#             marker=marker
#         ),
#         Line2D(
#             [0],
#             [0],
#             color=threads_color,
#             alpha=threads_alpha,
#             linestyle=threads_ls,
#             label="Max threads"
#         )
#     ];

    fig.legend( bbox_to_anchor=(1.5, 0.9));
    
    plt.show();
    
profile_chart2(prepped)